## Chatbot project with Heroku and Streamlit deployment

In this project we will build a very simple chatbot that will predict a tag/class from which the user input comes and from a list of possible responses randomly choose one to output. 

Our data is stored in the *intents* json file which we will load and preprocess before buidling a model.

In [51]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

In [52]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [53]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

The first preprocessing step will be to tokenize every sentence in patterns for each tag. This will basically means that each sentence will be split into words. Words from all the senteces will be stored together in the *words* list which will be used as a vocabulary. We also define a *documents* list which will store every pair of tokenized sentence and corresponding class as a tuple. Finally the *classes* list will contain all unique tags/classes.

In [4]:
words = []
classes = []
documents = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        documents.append((word, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
classes

['greeting',
 'goodbye',
 'thanks',
 'options',
 'adverse_drug',
 'blood_pressure',
 'blood_pressure_search',
 'pharmacy_search',
 'hospital_search']

In [6]:
words[:10]

['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey']

In [24]:
documents[:10]

[(['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hola'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye')]

Now we can *lemmatize* our *words* list ignoring some of the characters and also choose only the unique words to build a final vocabulary.

In [25]:
ignore_letters = ['!', '?', ',', '.']
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))

In [26]:
words[:10]

["'s",
 'a',
 'adverse',
 'all',
 'anyone',
 'are',
 'awesome',
 'be',
 'behavior',
 'blood']

We can now save *words* and *classes* lists as pickle files. They will be needed later on when we will build our chatbot.
To encode our patterns for training we can *lemmatize* each pattern and check at which postions words from our pattern appear in the vocabulary. At these positions we enter 1 and otherwise 0. This will encode every pattern as a sparse vector with many zeros. The target will be a class to which every pattern belongs encoded as an integer using a *classes_dict* which maps tags to indices.

In [27]:
classes_dict = {classes[i]: i for i in range(len(classes))}

In [28]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [29]:
training = []
for doc in documents:
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    bag = [1 if word in word_patterns else 0 for word in words]
    target = classes_dict[doc[1]]
    training.append([bag, target])

Now lets shuffle our training list and split for features and targets.

In [30]:
random.shuffle(training)
X = [pattern[0] for pattern in training]
y = [pattern[1] for pattern in training]

Our model will be a simple Sequential network with a *softmax* activation function at the last *Dense* layer. It will output probabilities for every tag for each pattern.

In [31]:
model = keras.models.Sequential([
    keras.layers.Dense(128, input_shape=[len(X[0])], activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(classes), activation='softmax')
])

In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               11264     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 9)                 585       
Total params: 20,105
Trainable params: 20,105
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, epochs=400, batch_size=4, verbose=1)

Epoch 1/400
12/12 [==============================] - 0s 1ms/step - loss: 2.1841 - accuracy: 0.1784
Epoch 2/400
12/12 [==============================] - 0s 818us/step - loss: 2.1455 - accuracy: 0.1001
Epoch 3/400
12/12 [==============================] - 0s 731us/step - loss: 2.1683 - accuracy: 0.2046
Epoch 4/400
12/12 [==============================] - 0s 728us/step - loss: 2.1347 - accuracy: 0.2129
Epoch 5/400
12/12 [==============================] - 0s 728us/step - loss: 2.0268 - accuracy: 0.2730
Epoch 6/400
12/12 [==============================] - 0s 818us/step - loss: 2.0145 - accuracy: 0.3101
Epoch 7/400
12/12 [==============================] - 0s 727us/step - loss: 1.9527 - accuracy: 0.3243
Epoch 8/400
12/12 [==============================] - 0s 727us/step - loss: 1.8750 - accuracy: 0.4375
Epoch 9/400
12/12 [==============================] - 0s 856us/step - loss: 1.8753 - accuracy: 0.5948
Epoch 10/400
12/12 [==============================] - 0s 818us/step - loss: 1.8162 - accuracy

12/12 [==============================] - 0s 818us/step - loss: 0.0129 - accuracy: 1.0000
Epoch 161/400
12/12 [==============================] - 0s 319us/step - loss: 0.0036 - accuracy: 1.0000
Epoch 162/400
12/12 [==============================] - 0s 818us/step - loss: 0.0123 - accuracy: 1.0000
Epoch 163/400
12/12 [==============================] - 0s 818us/step - loss: 0.0117 - accuracy: 1.0000
Epoch 164/400
12/12 [==============================] - 0s 818us/step - loss: 0.0601 - accuracy: 1.0000
Epoch 165/400
12/12 [==============================] - 0s 818us/step - loss: 0.0319 - accuracy: 0.9909
Epoch 166/400
12/12 [==============================] - 0s 818us/step - loss: 0.0104 - accuracy: 1.0000
Epoch 167/400
12/12 [==============================] - 0s 727us/step - loss: 0.0428 - accuracy: 1.0000
Epoch 168/400
12/12 [==============================] - 0s 727us/step - loss: 0.0043 - accuracy: 1.0000
Epoch 169/400
12/12 [==============================] - 0s 818us/step - loss: 0.0542 - a

12/12 [==============================] - 0s 727us/step - loss: 0.0033 - accuracy: 1.0000
Epoch 319/400
12/12 [==============================] - 0s 818us/step - loss: 0.0084 - accuracy: 1.0000
Epoch 320/400
12/12 [==============================] - 0s 818us/step - loss: 0.0084 - accuracy: 1.0000
Epoch 321/400
12/12 [==============================] - 0s 818us/step - loss: 0.0164 - accuracy: 0.9967
Epoch 322/400
12/12 [==============================] - 0s 727us/step - loss: 0.0030 - accuracy: 1.0000
Epoch 323/400
12/12 [==============================] - 0s 727us/step - loss: 0.0031 - accuracy: 1.0000
Epoch 324/400
12/12 [==============================] - 0s 909us/step - loss: 8.0033e-04 - accuracy: 1.0000
Epoch 325/400
12/12 [==============================] - 0s 727us/step - loss: 0.0017 - accuracy: 1.0000
Epoch 326/400
12/12 [==============================] - 0s 818us/step - loss: 0.0042 - accuracy: 1.0000
Epoch 327/400
12/12 [==============================] - 0s 818us/step - loss: 0.0046

There is also the other way to train our model. Instead of creating a vocabulary manually we can use a *tokenizer* from keras which will automatically create it and encode every sequence by index of each word in the vocabulary. Since the model will expect input sequences of the same length, we will have to pad every sequence to some maximum length.

In [28]:
classes = []
documents = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        documents.append((pattern, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [29]:
random.shuffle(documents)
X = [pattern[0] for pattern in documents]
y = [pattern[1] for pattern in documents]

In [30]:
vocab_size = 100
max_len = 20
tokenizer = keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X)

In [31]:
X_tokenized = tokenizer.texts_to_sequences(X)
X_tokenized = keras.preprocessing.sequence.pad_sequences(X_tokenized, maxlen=max_len)

In [89]:
y = np.array([classes_dict[tag] for tag in y])
embedding_dim = 20

Here we are going to use a simple RNN model with an Embedding layer.

In [97]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_shape=[None]),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(len(classes), activation='softmax')
])

In [98]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 20)          2000      
_________________________________________________________________
lstm (LSTM)                  (None, None, 64)          21760     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_15 (Dense)             (None, 9)                 297       
Total params: 36,473
Trainable params: 36,473
Non-trainable params: 0
_________________________________________________________________


In [99]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_tokenized, y, epochs=400, batch_size=4, verbose=1)

Epoch 1/400
12/12 [==============================] - 3s 8ms/step - loss: 2.2024 - accuracy: 0.0160
Epoch 2/400
12/12 [==============================] - 0s 7ms/step - loss: 2.1935 - accuracy: 0.2405
Epoch 3/400
12/12 [==============================] - 0s 7ms/step - loss: 2.1906 - accuracy: 0.0780
Epoch 4/400
12/12 [==============================] - 0s 7ms/step - loss: 2.1857 - accuracy: 0.1020
Epoch 5/400
12/12 [==============================] - 0s 8ms/step - loss: 2.1767 - accuracy: 0.1531
Epoch 6/400
12/12 [==============================] - 0s 9ms/step - loss: 2.1458 - accuracy: 0.2379
Epoch 7/400
12/12 [==============================] - 0s 8ms/step - loss: 2.1352 - accuracy: 0.1306
Epoch 8/400
12/12 [==============================] - 0s 8ms/step - loss: 2.0819 - accuracy: 0.2687
Epoch 9/400
12/12 [==============================] - 0s 7ms/step - loss: 2.0356 - accuracy: 0.2749
Epoch 10/400
12/12 [==============================] - 0s 7ms/step - loss: 1.8977 - accuracy: 0.2792
Epoch 11/

12/12 [==============================] - 0s 8ms/step - loss: 0.0144 - accuracy: 1.0000
Epoch 84/400
12/12 [==============================] - 0s 8ms/step - loss: 0.0142 - accuracy: 1.0000
Epoch 85/400
12/12 [==============================] - 0s 7ms/step - loss: 0.0155 - accuracy: 1.0000
Epoch 86/400
12/12 [==============================] - 0s 7ms/step - loss: 0.0138 - accuracy: 1.0000
Epoch 87/400
12/12 [==============================] - 0s 8ms/step - loss: 0.0135 - accuracy: 1.0000
Epoch 88/400
12/12 [==============================] - 0s 8ms/step - loss: 0.0138 - accuracy: 1.0000
Epoch 89/400
12/12 [==============================] - 0s 8ms/step - loss: 0.0121 - accuracy: 1.0000
Epoch 90/400
12/12 [==============================] - 0s 8ms/step - loss: 0.0120 - accuracy: 1.0000
Epoch 91/400
12/12 [==============================] - 0s 8ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 92/400
12/12 [==============================] - 0s 7ms/step - loss: 0.0139 - accuracy: 1.0000
Epoch 93/400


12/12 [==============================] - 0s 8ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 246/400
12/12 [==============================] - 0s 8ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 247/400
12/12 [==============================] - 0s 8ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 248/400
12/12 [==============================] - 0s 7ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 249/400
12/12 [==============================] - 0s 7ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 250/400
12/12 [==============================] - 0s 8ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 251/400
12/12 [==============================] - 0s 7ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 252/400
12/12 [==============================] - 0s 7ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 253/400
12/12 [==============================] - 0s 7ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 254/400
12/12 [==============================] - 0s 7ms/step - loss: 0.0015 - accuracy: 1.0000
Epoc

Before moving on to the app we can also look at the another way of training our model. Instead of creating a vocabulary and encoding every pattern based on this vocabulary, we can use *tf-idf* vectorizer transformer from sklearn which will convert each word in every pattern to its relative frequency in the entire corpus. Then we also encode our labels using a *LabelEncoder*. Both transformers are saved so that we can we use them later.

In [54]:
classes = []
documents = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        documents.append((pattern, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [55]:
random.shuffle(documents)
X = [pattern[0] for pattern in documents]
y = [pattern[1] for pattern in documents]

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

lencoder= LabelEncoder()
y = lencoder.fit_transform(y)

In [57]:
print(vectorizer.get_feature_names())

['adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'details', 'do', 'dont', 'drug', 'drugs', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacies', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'results', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [58]:
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))
pickle.dump(lencoder, open('lencoder.pkl', 'wb'))

In [66]:
model = keras.models.Sequential([
    keras.layers.Dense(128, input_shape=[len(vectorizer.get_feature_names())], activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(classes), activation='softmax')
])

In [67]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 128)               11264     
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 9)                 585       
Total params: 20,105
Trainable params: 20,105
Non-trainable params: 0
_________________________________________________________________


In [68]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X.todense(), y, epochs=400, batch_size=4, verbose=1)

Epoch 1/400
12/12 [==============================] - 0s 996us/step - loss: 2.1802 - accuracy: 0.0803
Epoch 2/400
12/12 [==============================] - 0s 909us/step - loss: 2.1716 - accuracy: 0.1729
Epoch 3/400
12/12 [==============================] - 0s 820us/step - loss: 2.1783 - accuracy: 0.0918
Epoch 4/400
12/12 [==============================] - 0s 818us/step - loss: 2.1484 - accuracy: 0.2449
Epoch 5/400
12/12 [==============================] - 0s 727us/step - loss: 2.1271 - accuracy: 0.2725
Epoch 6/400
12/12 [==============================] - 0s 727us/step - loss: 2.0903 - accuracy: 0.2937
Epoch 7/400
12/12 [==============================] - 0s 818us/step - loss: 2.0542 - accuracy: 0.3084
Epoch 8/400
12/12 [==============================] - 0s 727us/step - loss: 2.0009 - accuracy: 0.5520
Epoch 9/400
12/12 [==============================] - 0s 727us/step - loss: 2.0179 - accuracy: 0.4603
Epoch 10/400
12/12 [==============================] - 0s 727us/step - loss: 1.9908 - accura

12/12 [==============================] - 0s 818us/step - loss: 0.0860 - accuracy: 0.9387
Epoch 82/400
12/12 [==============================] - 0s 818us/step - loss: 0.0706 - accuracy: 1.0000
Epoch 83/400
12/12 [==============================] - 0s 727us/step - loss: 0.0717 - accuracy: 1.0000
Epoch 84/400
12/12 [==============================] - 0s 727us/step - loss: 0.0548 - accuracy: 1.0000
Epoch 85/400
12/12 [==============================] - 0s 727us/step - loss: 0.0439 - accuracy: 1.0000
Epoch 86/400
12/12 [==============================] - 0s 728us/step - loss: 0.0541 - accuracy: 1.0000
Epoch 87/400
12/12 [==============================] - 0s 727us/step - loss: 0.0227 - accuracy: 1.0000
Epoch 88/400
12/12 [==============================] - 0s 727us/step - loss: 0.0946 - accuracy: 1.0000
Epoch 89/400
12/12 [==============================] - 0s 727us/step - loss: 0.0478 - accuracy: 1.0000
Epoch 90/400
12/12 [==============================] - 0s 818us/step - loss: 0.1312 - accuracy: 

12/12 [==============================] - 0s 818us/step - loss: 0.0116 - accuracy: 1.0000
Epoch 161/400
12/12 [==============================] - 0s 818us/step - loss: 0.0111 - accuracy: 1.0000
Epoch 162/400
12/12 [==============================] - 0s 818us/step - loss: 0.0076 - accuracy: 1.0000
Epoch 163/400
12/12 [==============================] - 0s 818us/step - loss: 0.0108 - accuracy: 1.0000
Epoch 164/400
12/12 [==============================] - 0s 727us/step - loss: 0.0645 - accuracy: 1.0000
Epoch 165/400
12/12 [==============================] - 0s 818us/step - loss: 0.0096 - accuracy: 1.0000
Epoch 166/400
12/12 [==============================] - 0s 818us/step - loss: 0.0199 - accuracy: 1.0000
Epoch 167/400
12/12 [==============================] - 0s 818us/step - loss: 0.0319 - accuracy: 1.0000
Epoch 168/400
12/12 [==============================] - 0s 727us/step - loss: 0.0643 - accuracy: 1.0000
Epoch 169/400
12/12 [==============================] - 0s 818us/step - loss: 0.0300 - a

12/12 [==============================] - 0s 818us/step - loss: 0.0076 - accuracy: 1.0000
Epoch 240/400
12/12 [==============================] - 0s 812us/step - loss: 0.0135 - accuracy: 1.0000
Epoch 241/400
12/12 [==============================] - 0s 818us/step - loss: 0.0046 - accuracy: 1.0000
Epoch 242/400
12/12 [==============================] - 0s 818us/step - loss: 0.0076 - accuracy: 1.0000
Epoch 243/400
12/12 [==============================] - 0s 818us/step - loss: 0.0026 - accuracy: 1.0000
Epoch 244/400
12/12 [==============================] - 0s 727us/step - loss: 0.0178 - accuracy: 1.0000
Epoch 245/400
12/12 [==============================] - 0s 818us/step - loss: 0.0090 - accuracy: 1.0000
Epoch 246/400
12/12 [==============================] - 0s 818us/step - loss: 0.0173 - accuracy: 1.0000
Epoch 247/400
12/12 [==============================] - 0s 818us/step - loss: 0.0315 - accuracy: 0.9826
Epoch 248/400
12/12 [==============================] - 0s 727us/step - loss: 0.0031 - a

12/12 [==============================] - 0s 818us/step - loss: 0.0029 - accuracy: 1.0000
Epoch 319/400
12/12 [==============================] - 0s 727us/step - loss: 0.0028 - accuracy: 1.0000
Epoch 320/400
12/12 [==============================] - 0s 727us/step - loss: 0.0040 - accuracy: 1.0000
Epoch 321/400
12/12 [==============================] - 0s 818us/step - loss: 0.0010 - accuracy: 1.0000
Epoch 322/400
12/12 [==============================] - 0s 818us/step - loss: 0.0402 - accuracy: 1.0000
Epoch 323/400
12/12 [==============================] - 0s 818us/step - loss: 0.0037 - accuracy: 1.0000
Epoch 324/400
12/12 [==============================] - 0s 818us/step - loss: 0.0015 - accuracy: 1.0000
Epoch 325/400
12/12 [==============================] - 0s 818us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 326/400
12/12 [==============================] - 0s 818us/step - loss: 0.0075 - accuracy: 1.0000
Epoch 327/400
12/12 [==============================] - 0s 727us/step - loss: 0.0073 - a

12/12 [==============================] - 0s 909us/step - loss: 7.8357e-04 - accuracy: 1.0000
Epoch 398/400
12/12 [==============================] - 0s 1ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 399/400
12/12 [==============================] - 0s 727us/step - loss: 0.0021 - accuracy: 1.0000
Epoch 400/400
12/12 [==============================] - 0s 909us/step - loss: 8.0353e-04 - accuracy: 1.0000


Now we can save our model and use it later to build the chatbot.

In [70]:
model.save('chatbot_model.h5', model)